# NLP

In [6]:
from transformers import BertForQuestionAnswering, AutoTokenizer, DefaultDataCollator, TrainingArguments, Trainer
import torch
from nltk.stem import PorterStemmer
import re

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
re.sub(r'[^\w\s\d]', '', 'HELLO!122 ')

'HELLO122 '

### Pre-Processing

These are the steps that I added:

1) Stop Word Removal
2) Stemming(Reducing words to its root word)
3) Tokenization(Change it into index)(Can explore embedding like Glover or Word2VeC, embedder basically tokenize the work in a way that represent the relation/meaning of the word)
4) 

In [15]:
def preprocess_function(examples):
    #Regex to remove punctuations
    question_string = re.sub(r'[^\w\s\d]', '', examples["question"])
    # Stemming and lower casing the words
    porter = PorterStemmer()
    questions = [porter.stem(q.strip().lower()) for q in question_string]
    #tokenization
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=128,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    # IDK WHAT HAPPENING HERE, I JUST KEPT THEIR CODE:<, i think has to do with the 
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_data = data.map(preprocess_function, batched=True)
data_collator = DefaultDataCollator()

NameError: name 'data' is not defined